# Text classification on movie reviews Dataset from stanford with Bert

In [ ]:
fig = dict({
    "data": [{"type": "bar",
              "x": [1, 2, 3],
              "y": [1, 3, 2]}],
    "layout": {"title": {"text": "A Figure Specified By Python Dictionary"}}
})

# To display the figure defined by this dict, use the low-level plotly.io.show function
import plotly.io as pio

pio.show(fig)

In [2]:
!mkdir  /content/drive/MyDrive/transformers_for_nlp
%cd ./drive/MyDrive/transformers_for_nlp
#from google.colab import files 
#files.upload()

/content/drive/MyDrive/transformers_for_nlp


In [3]:
!pip install transformers


     |████████████████████████████████| 2.1MB 14.5MB/s 
     |████████████████████████████████| 3.3MB 40.2MB/s 
     |████████████████████████████████| 870kB 52.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=945a24c319af7633cb914104a249bcfc15131f5bee9afe9879604b723742058b
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
# Downloading the dataset.
!wget -q -nc http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz


In [5]:
# Unziping the dataset.
!tar -zxf /content/drive/MyDrive/transformers_for_nlp/aclImdb_v1.tar.gz

In [7]:
!pip install -q git+https://github.com/gmihaila/ml_things.git


     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 10.3MB 21.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [8]:
import os
import io
from tqdm.notebook import tqdm
from torch.utils.data import Dataset , DataLoader 
from ml_things import plot_dict , fix_text , plot_confusion_matrix
from sklearn.metrics import classification_report , accuracy_score
from transformers import (AutoConfig ,AutoModelForSequenceClassification ,AutoTokenizer,AdamW,
                          get_linear_schedule_with_warmup , set_seed)


In [12]:
import torch
from torch import nn 
import torch.nn.functional as F

In [19]:
set_seed(26)
epochs = 4
batches =  32 # as max seq length is small but for max _Seq_len =512 batch_Size should be very small (due togpu issues)
max_seq_len = 50
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cpu' :
    print("Using cpu ,This will b slow")
model_name = 'bert_base_cased'

labels = { "pos" : 1 , "neg" : 0}
n_labels = len(labels)



In [17]:
class DataSet(Dataset):
    r"""
    This class can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.
 
  Arguments:
 
    path (:obj:`str`):
        Path to the data partition.
     
    tokenizer (:obj:`transformers.tokenization_?`):
        Transformer type tokenizer used to process raw text into numbers.
 
    labels (:obj:`dict`):
        Dictionary to encode any labels names into numbers. Keys map to 
        labels names and Values map to number associated to those labels.
 
    max_sequence_len (:obj:`int`, `optional`)
        Value to indicate the maximum desired sequence to truncate or pad text
        sequences. If no value is passed it will used maximum sequence size
        supported by the tokenizer and model.
     """
    def _init__(self , path , tokenizer , labels , _max_seq_len):
        super(DataSet,self).__init__()
        if not os.path.isdir(path):
             print("Path to directory does not exist.")

        texts = []
        labels = []

        #the labels are defined by folders with data we loop through each label.

        for label , label_id in tqdm(labels.items()):
             text_path  = os.path.join(path , label) #path/pos | path/neg

             all_file_names = os.listdir(text_path)[:10] 
             for file_name in tqdm(all_file_names):
                 file_path = os.path.join(text_path , file_name)

                 text = io.open(file_path , mode = 'r' , encoding = 'utf-8').read()
                 text = fix_text(text) #solves any encoding issues
                 texts.append(text)
                 labels.append(label_id)
        self.n = len(labels)
        #using tokenizer on text ,returns a dictionary with tokenized integer ids under input_ids key
        self.tokenized_inputs = tokenizer(texts ,add_special_tokens = True , truncation = True, padding = True )
        self.seq_len = self.tokenized_inputs['input_ids'].shape[-1] #length of sequence

        self.tokenized_inputs.update({"labels" : torch.tensor(labels)})
        print(self.tokenized_inputs)
        def __len__(self) :
            return self.n

        def __getitem__(self , index):
            r"""
            Arguments:
 
            index (:obj:`int`):
                Index position to pick an example to return.
        
            Returns:
            :obj:`Dict[str, object]`: Dictionary of inputs that feed into the model.
            It holddes the statement `model(**Returned Dictionary)`.

            """
            return {key : self.tokenized_inputs[key][items] for key in self.tokenized_inputs.keys()}







In [20]:
d = Dataset("/content/drive/MyDrive/transformers_for_nlp/aclImdb/train" , AutoTokenizer , labels , max_seq_len)

TypeError: ignored